In [18]:
!pip install tensorboard --quiet
!pip install matplotlib --quiet
!pip install scikit-learn --quiet
!pip install torchsummary --quiet
!pip install opencv-python --quiet
!pip install monai --quiet

In [19]:
import sys
import time
import torch
from torch.utils.tensorboard import SummaryWriter
from datetime import date
import json

import io
import s3fs

from dataloader_sep import *
from model.model import * 
from trainer import * 

import os
print(os.getcwd())

/home/onyxia/work/Nanowire_image_segmentation


In [20]:
device = torch.device("cuda") 

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

# BUCKET = "ldegeorge/images"
# fs.ls(BUCKET)

In [ ]:
        with fs.open(path, 'rb') as file:
            image_data = file.read()

In [4]:
image_paths = []
mask_paths = []

bucket = "ldegeorge/labeled_data_pt"
for path in fs.ls(bucket):
    if path == 'ldegeorge/images/.keep':
        pass
    else:
        image_paths.append(path)

bucket = "ldegeorge/binary_masks_pt"
for path in fs.ls(bucket):
    if path == 'ldegeorge/binary_masks/.keep':
        pass
    else:
        mask_paths.append(path)

class train_LabeledDataset(torch.utils.data.Dataset):
    def __init__(self, image_list, mask_list, transform=None):
        self.image_list = image_list
        self.mask_list = mask_list

    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, index):
        image = self.image_list[index]
        mask = self.mask_list[index]
        with fs.open(image, 'rb') as file:
            image_data = file.read()
        with fs.open(mask, 'rb') as file:
            mask_data = file.read()
        mask = torch.load(mask_data)
        return image, mask

class eval_LabeledDataset(torch.utils.data.Dataset):
    def __init__(self, image_list, mask_list, transform=None):
        self.image_list = image_list
        self.mask_list = mask_list

    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, index):
        image = self.image_list[index]
        mask = self.mask_list[index]
        with fs.open(image, 'rb') as file:
            image_data = file.read()
        with fs.open(mask, 'rb') as file:
            mask_data = file.read()
        mask = torch.load(mask_data)
        return image, mask

batch_size = 4

train_images, eval_images, train_masks, eval_masks = train_test_split(image_list, mask_list, test_size=0.2)
train_dataset = train_LabeledDataset(train_images, train_masks)
eval_dataset = eval_LabeledDataset(eval_images, eval_masks)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)

dataloader ok


In [22]:
len(dataloader)

NameError: name 'dataloader' is not defined

In [5]:
def main():

    torch.cuda.empty_cache()
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    print("start training")
    start_time = time.time()
    lr = 0.0002

    generator = UnetGenerator().to(device)
    discriminator = ConditionalDiscriminator().to(device)
    criterions = [ GeneratorLoss(alpha=100), DiscriminatorLoss() ]

    trainer = pix2pix_trainer(generator, discriminator, criterions, lr, batch_size, timestamp=timestamp, dataloader=dataloader)
    trainer.train()

    print("end of training")

In [ ]:
main()

start training
56
EPOCH 0:
  batch 0 loss: [62.662261962890625, 0.7088532447814941]
  batch 2 loss: [53.87037467956543, 0.8901240825653076]
  batch 4 loss: [46.6334342956543, 0.7785354256629944]
  batch 6 loss: [46.6695499420166, 0.6993644535541534]
  batch 8 loss: [45.96317481994629, 0.6947112381458282]
  batch 10 loss: [36.016408920288086, 0.6940074563026428]
  batch 12 loss: [27.848180770874023, 0.6937751770019531]
  batch 14 loss: [36.485618591308594, 0.6936433613300323]
  batch 16 loss: [43.35696029663086, 0.6934984028339386]
  batch 18 loss: [45.3878231048584, 0.6935330629348755]
  batch 20 loss: [33.673513412475586, 0.6933387517929077]
  batch 22 loss: [33.71762561798096, 0.6933234632015228]
  batch 24 loss: [40.779977798461914, 0.6933051645755768]
  batch 26 loss: [34.26003837585449, 0.6932120323181152]
  batch 28 loss: [37.25369644165039, 0.693175733089447]
  batch 30 loss: [35.46072006225586, 0.6932038068771362]
  batch 32 loss: [43.36826515197754, 0.693413257598877]
  batch 